In [3]:
!pip install onnx onnxruntime

In [1]:
import onnx
import onnxruntime as ort
import numpy as np

# Define input and output tensor names
input1_name = "X"
input2_name = "L"
input3_name = "M"
clip_output_name = "Y"


# Create the ONNX model with Clip operator
def create_clip_model(input_rank, dtype):

    #Create "input-rank" input tensors and 2 scalars (min and max values for clipping)
    input1 = onnx.helper.make_tensor_value_info(input1_name, dtype, input_rank)
    input2 = onnx.helper.make_tensor_value_info(input2_name, dtype, [])
    input3 = onnx.helper.make_tensor_value_info(input3_name, dtype, [])

    # Create output tensor (final result after clip operation)
    clip_output = onnx.helper.make_tensor_value_info(clip_output_name, dtype, input_rank)

    # Define clip node
    clip_node = onnx.helper.make_node(
        "Clip",
        inputs=[input1_name, input2_name, input3_name],
        outputs=[clip_output_name],
    )

    # Create the ONNX graph
    graph_def = onnx.helper.make_graph(
        [clip_node],
        "Clip",
        [input1, input2, input3],
        [clip_output],
    )

    # Create the ONNX model
    model = onnx.helper.make_model(graph_def, opset_imports=[onnx.helper.make_opsetid("", 13)]) # Explicitly set opset to 13
    model.ir_version = 10 
    onnx.checker.check_model(model)

    # Save the model
    onnx.save(model, "clip.onnx")

    # Load and run the model with ONNX Runtime
    session = ort.InferenceSession("clip.onnx")
    return session

def do_clip(x, l, m, session):
    # Run inference
    output = session.run(None, {input1_name: x, input2_name: l, input3_name: m})

    x_f = (np.array2string(x, separator=',', max_line_width=np.inf).replace('\n', ''))
    l_f = (np.array2string(l, separator=',', max_line_width=np.inf).replace('\n', ''))
    m_f = (np.array2string(m, separator=',', max_line_width=np.inf).replace('\n', ''))
    y_f = (np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', ''))

    # Display results
    print(f"X={x_f}, L={l_f}, M={m_f}")
    print(f"Result = {y_f}")


np.set_printoptions(precision=None, floatmode='fixed')

## Nominal Cases

In [2]:
# Case N1: 1-rank tensor (int32), 2 scalars (int32)
onnx_type = onnx.TensorProto.INT32
x = np.array([-2, -1, 0, 1, 2], dtype=np.int32)
l = np.array(-1, dtype=np.int32)
m = np.array(1, dtype=np.int32)
session = create_clip_model([None], onnx_type)
do_clip(x, l, m, session)

X=[-2,-1, 0, 1, 2], L=-1, M=1
Result = [-1,-1, 0, 1, 1]


In [3]:
# Case N2: 2-rank tensor (int32), 2 scalars (int32)
onnx_type = onnx.TensorProto.INT32
x = np.array([[-20,-10,0,10,20],[-15,-10,5,10,0]], dtype=np.int32)
l = np.array(-1, dtype=np.int32)
m = np.array(1, dtype=np.int32)
session = create_clip_model([None, None], onnx_type)
do_clip(x, l, m, session)

X=[[-20,-10,  0, 10, 20], [-15,-10,  5, 10,  0]], L=-1, M=1
Result = [[-1,-1, 0, 1, 1], [-1,-1, 1, 1, 0]]


In [4]:
# Case N3: 1-rank tensor (float32), 2 scalars (float32)
onnx_type = onnx.TensorProto.FLOAT
x = np.array([-2.0, -1.0, 0.0, 1.0, 2.0], dtype=np.float32)
l = np.array(-1.0, dtype=np.float32)
m = np.array(1.0, dtype=np.float32)
session = create_clip_model([None], onnx_type)
do_clip(x, l, m, session)

X=[-2.00000000,-1.00000000, 0.00000000, 1.00000000, 2.00000000], L=-1.00000000, M=1.00000000
Result = [-1.00000000,-1.00000000, 0.00000000, 1.00000000, 1.00000000]


In [5]:
# Case N4: 2-rank tensor (float32), 2 scalars (float32)
onnx_type = onnx.TensorProto.FLOAT
x = np.array([[-2.0, -1.0, 0.0, 1.0, 2.0],[-5.0, -2.5, 0.0, 2.5, 5.0]], dtype=np.float32)
l = np.array(-3.0, dtype=np.float32)
m = np.array(3.0, dtype=np.float32)
session = create_clip_model([None,None], onnx_type)
do_clip(x, l, m, session)


X=[[-2.00000000,-1.00000000, 0.00000000, 1.00000000, 2.00000000], [-5.00000000,-2.50000000, 0.00000000, 2.50000000, 5.00000000]], L=-3.00000000, M=3.00000000
Result = [[-2.00000000,-1.00000000, 0.00000000, 1.00000000, 2.00000000], [-3.00000000,-2.50000000, 0.00000000, 2.50000000, 3.00000000]]


In [6]:
# Case N5: 2-rank tensor (float32), 2 scalars (float32), min > max
onnx_type = onnx.TensorProto.FLOAT
x = np.array([[-2.0, -1.0, 0.0, 1.0, 2.0],[-5.0, -2.5, 0.0, 2.5, 5.0]], dtype=np.float32)
l = np.array(10.0, dtype=np.float32)
m = np.array(5.0, dtype=np.float32)
session = create_clip_model([None,None], onnx_type)
do_clip(x, l, m, session)

X=[[-2.00000000,-1.00000000, 0.00000000, 1.00000000, 2.00000000], [-5.00000000,-2.50000000, 0.00000000, 2.50000000, 5.00000000]], L=10.00000000, M=5.00000000
Result = [[5.00000000,5.00000000,5.00000000,5.00000000,5.00000000], [5.00000000,5.00000000,5.00000000,5.00000000,5.00000000]]


## No nominal cases (nan and inf values)

In [7]:
# Case N1: 1-rank tensor (float32), 2 scalars (float32) -  Nan values in input tensor
onnx_type = onnx.TensorProto.FLOAT
x = np.array([-2.0, -1.0, 0.0, 1.0, np.nan], dtype=np.float32)
l = np.array(-1.0, dtype=np.float32)
m = np.array(1.0, dtype=np.float32)
session = create_clip_model([None], onnx_type)
do_clip(x, l, m, session)

X=[-2.00000000,-1.00000000, 0.00000000, 1.00000000,        nan], L=-1.00000000, M=1.00000000
Result = [-1.00000000,-1.00000000, 0.00000000, 1.00000000,        nan]


In [8]:
# Case N2: 1-rank tensor (float32), 2 scalars (float32) -  Nan values in L
onnx_type = onnx.TensorProto.FLOAT
x = np.array([-2.0, -1.0, 0.0, 1.0, 2.0], dtype=np.float32)
l = np.array(np.nan, dtype=np.float32)
m = np.array(1.0, dtype=np.float32)
session = create_clip_model([None], onnx_type)
do_clip(x, l, m, session)

X=[-2.00000000,-1.00000000, 0.00000000, 1.00000000, 2.00000000], L=nan, M=1.00000000
Result = [-2.00000000,-1.00000000, 0.00000000, 1.00000000, 1.00000000]


In [9]:
# Case N3: 1-rank tensor (float32), 2 scalars (float32) -  Nan values in M
onnx_type = onnx.TensorProto.FLOAT
x = np.array([-2.0, -1.0, 0.0, 1.0, 2.0], dtype=np.float32)
l = np.array(-1, dtype=np.float32)
m = np.array(np.nan, dtype=np.float32)
session = create_clip_model([None], onnx_type)
do_clip(x, l, m, session)

X=[-2.00000000,-1.00000000, 0.00000000, 1.00000000, 2.00000000], L=-1.00000000, M=nan
Result = [-1.00000000,-1.00000000, 0.00000000, 1.00000000, 2.00000000]


In [10]:
# Case N4: 1-rank tensor (float32), 2 scalars (float32) -  Nan values in L and M
onnx_type = onnx.TensorProto.FLOAT
x = np.array([-2.0, -1.0, 0.0, 1.0, 2.0], dtype=np.float32)
l = np.array(np.nan, dtype=np.float32)
m = np.array(np.nan, dtype=np.float32)
session = create_clip_model([None], onnx_type)
do_clip(x, l, m, session)

X=[-2.00000000,-1.00000000, 0.00000000, 1.00000000, 2.00000000], L=nan, M=nan
Result = [-2.00000000,-1.00000000, 0.00000000, 1.00000000, 2.00000000]
